# Generate EMA using daily data service - Python

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *exponential moving average - EMA* from a `daily data service`.

### Inputs/outputs
Exponential moving averages' requests require instrument's identifier, date time intervals and number of points as per inputs.

EMA reduces the lag by applying greater weight to recent data points.The weighting applied to the most recent price depends on the number of points in the moving average; The smaller the point count is, the more weight is applied to the most recent price.

N-points EMA data point is computed using the previous' *EMA* value and the wighting multiplier as follows:

$$ EMA = (Close-EMAp)*k+EMAp $$

Where:
* *Price* is the close price
* *EMAp* is the previous points' *EMA* value, *SMA* is used as the EMAp in the first calculation
* *N* is the count of points
* *$ k = \frac{2}{N+1} $* is the *Weighting Multiplier*

<div class="alert alert-block alert-info">
    <b>Note:</b> <i>EMA</i> adapts quickly to price changes than the <i>SMA</i>, when a price reverses direction, the <i>EMA</i> will reverse direction quicker than the <i>SMA</i> due to the weighting 
</div>

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve EMA indicator from the hosted service. The queried endpoint in this script are:
* *DailyEmaSerice*: to directly retrieve daily EMA data from the server


### Modules required
1. Systemathics:
    * *systemathics.apis.services.daily_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package

***

# Run daily EMA sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.daily_analytics.v1.daily_ema_pb2 as daily_ema
import systemathics.apis.services.daily_analytics.v1.daily_ema_pb2_grpc as daily_ema_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *daily ema* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The EMA parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 EMA parameters

In [ ]:
# set EMAs lengths
long_ema = 200;
short_ema = 50;

#### 3.3 Time period selection

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2019, month = 1, day = 1),
    end_date = date.Date(year = 2020, month = 12, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

#### 3.4 EMAs request creation
The following code snippet creates *gRPC client*, process daily EMA requests and returns the reply.

In [ ]:
# generate daily long EMA request
long_ema_request = daily_ema.DailyEmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    length = long_ema,
    constraints = constraints
)

# generate daily short EMA request
short_ema_request = daily_ema.DailyEmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    length = short_ema,
    constraints = constraints
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the daily ema service
    service = daily_ema_service.DailyEmaServiceStub(channel)
    
    # process the daily long EMA request
    long_ema_response = service.DailyEma(
        request = long_ema_request, 
        metadata = [('authorization', token)]
    )
    
    # process the daily short EMA request
    short_ema_response = service.DailyEma(
        request = short_ema_request, 
        metadata = [('authorization', token)]
    )

The following code snippet displays total data points count:

In [ ]:
print("Total data points retrieved for long EMA: ", len(long_ema_response.data))
print("Total data points retrieved for short EMA: ", len(short_ema_response.data))

### Step 4: Visualize data

#### 4.1 Retrieve EMAs data
In the following code snippets, the daily EMA reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# prepare the emas dataframe content
dates = [datetime(l.date.year, l.date.month, l.date.day) for l in long_ema_response.data]
prices = [l.value for l in long_ema_response.data]
long_ema_data = [l.average for l in long_ema_response.data]
short_ema_data = [s.average for s in short_ema_response.data]

In [ ]:
# create a pandas dataframe with dates, prices and emas
d = {'Date': dates, 'Short ema': short_ema_data, 'Long ema': long_ema_data, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot EMAs and daily prices
Plot EMA and daily prices request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Long ema'], mode='lines', name='{0}-days EMA'.format(long_ema)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Short ema'], mode='lines', name='{0}-days EMA'.format(short_ema)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], mode='lines', name='Price'))

fig.update_layout(title = 'Long & short EMA for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Price')
fig.show()